**Import necessary packages/modules**

In [ ]:
# Cell 1
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
from matplotlib.ticker import AutoMinorLocator

**Connect this notebook to your Google Drive**

In [ ]:
# Cell 2
drive.mount("/content/gdrive", force_remount=True)
notebook_path = Path("/content/gdrive/MyDrive/wdts-psi")
notebook_path /= Path("Session 07 - Advanced Algorithms")
notebook_path

**Define a function to calculate the Discrete Fourier Transform**
1. We will use Euler's Formula $e^{i\theta}=\cos\theta + \sin\theta$
2. Notice the first complex term is NOT multipled by 2

In [ ]:
# Cell 3
def dft(ts, ys):
    num_samples = ts.size  # ts = sample time
    num_terms = int(num_samples / 2)  # Nyquist limit
    # ct = complex terms of the DFT
    ct = np.zeros(num_terms, dtype=complex)
    for k in range(0, num_terms):  # k = filter wave number
        for n in range(0, num_samples):  # n = sample number
            ct[k] += ys[n] * np.exp(complex(0, (k * ts[n])))
    ct = ct * 2 / num_samples
    ct[0] /= 2  # DC value should NOT be doubled
    return ct

**Define a function to calculate the Inverse Discrete Fourier Trasform**

In [ ]:
# Cell 4
def idft(ts, ct):
    num_samples = ts.size  # ts = sample time
    num_terms = ct.size
    # yr = reconstructed y values
    yr = np.zeros(num_samples, dtype=complex)
    for n in range(0, num_samples):  # n = sample number
        for k in range(0, num_terms):  # k = filter wave number
            yr[n] += ct[k] * np.exp(complex(0, -(k * ts[n])))
    return yr

**Define a function to plot the samples read from the data file**

In [ ]:
# Cell 5
def plot_samples(ax, ts, ys):
    num_samples = ts.size
    ax.plot(ts, ys, color="lightgray", linewidth=1)
    ax.scatter(ts, ys, color="black", marker=".", s=10.0, zorder=2)
    ax.set_title(f"Sampled Wave ({num_samples} samples)")
    ax.set_xlabel("scaled time", loc="right")
    ax.set_ylabel("amplitude")

**Define a function to plot the DFT (frequency histogram)**

In [ ]:
# Cell 6
def plot_dft(ax, ct):
    num_terms = 40

    # fmt: off
    ax.bar(range(0, num_terms), ct.real[:num_terms],
        color="blue", label="cosine", zorder=2)
    ax.bar(range(0, num_terms), ct.imag[:num_terms],
        color="red",  label="sine", zorder=2)
    # fmt: on

    ax.grid(which="major", axis="x", color="black", linewidth=1)
    ax.grid(which="minor", axis="x", color="lightgray", linewidth=1)
    ax.grid(which="major", axis="y", color="black", linewidth=1)
    ax.grid(which="minor", axis="y", color="lightgray", linewidth=1)

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.set_title("Discrete Fourier Transform")
    ax.set_xlabel("frequency", loc="right")
    ax.set_ylabel("amplitude")
    ax.legend(loc="upper right")

**Define a function to plot the IDFT**

In [ ]:
# Cell 7
def plot_idft(ax, ts, yr):
    num_samples = ts.size
    ax.plot(ts, yr, color="purple")
    ax.set_title(f"Inverse DFT ({num_samples} samples)")
    ax.set_xlabel("scaled time", loc="right")
    ax.set_ylabel("amplitude")

**Define a function to plot the Power Spectrum of the DFT**

In [ ]:
# Cell 8
def plot_power_spectrum(ax, ct):
    num_terms = 40
    ax.bar(
        range(0, num_terms), abs(ct[:num_terms]), color="green", label="sine", zorder=2
    )
    ax.grid(which="major", axis="x", color="black", linewidth=1)
    ax.grid(which="minor", axis="x", color="lightgray", linewidth=1)
    ax.grid(which="major", axis="y", color="black", linewidth=1)
    ax.grid(which="minor", axis="y", color="lightgray", linewidth=1)

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.set_title("Power Spectrum")
    ax.set_xlabel("frequency", loc="right")
    ax.set_ylabel(r"$\Vert amplitude \Vert$")

**Define a function to analyze the samples within the given data file name**

In [ ]:
# Cell 9
def analyze_samples(file_name):
    file_path = notebook_path / file_name
    samples = np.genfromtxt(file_path, delimiter=",")
    ts, ys = samples.T  # ts = sample time, ys = sample value

    ct = dft(ts, ys)  # ct = complex terms of the DFT
    yr = idft(ts, ct)  # yr = reconstructed y values

    plt.figure(file_name, figsize=(12, 8))

    plot_samples(plt.subplot(2, 2, 1), ts, ys)
    plot_dft(plt.subplot(2, 2, 2), ct)
    plot_idft(plt.subplot(2, 2, 3), ts, np.real(yr))
    plot_power_spectrum(plt.subplot(2, 2, 4), ct)

    plt.tight_layout()
    plt.show()

analyze_samples("samples.csv")